<a href="https://colab.research.google.com/github/aa10sh/demorepo/blob/main/Phone_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Radhe Radhe")

Radhe Radhe


# Data Preprocessing

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [38]:
df=pd.read_csv("mobile_prices_2023.csv")

# Let's observe the datset mannually first.
1. multiple corrupted values are present in battery col.
2. Date of scrapping is same for entire dataset.
3. Storage is NAN for all the iPhones

In [4]:
df.sample(3)

,Phone Name,Rating ?/5,Number of Ratings,RAM,ROM/Storage,Back/Rare Camera,Front Camera,Battery,Processor,Price in INR,Date of Scraping
228,"realme Narzo 30 (Racing Blue, 64 GB)",4.3,"77,321",6 GB RAM,64 GB ROM,48MP + 2MP + 2MP,16MP Front Camera,5000 mAh,MediaTek Helio G95 Processor,"₹11,499",6/17/2023
649,"SAMSUNG Galaxy S23 Plus 5G (Phantom Black, 256...",4.5,140,8 GB RAM,256 GB ROM,50MP + 10MP + 12MP,12MP Front Camera,4700 mAh,Qualcomm Snapdragon 8 Gen 2 Processor,"₹94,999",6/17/2023
1620,"OnePlus Nord CE 5G (Blue Void, 128 GB)",4.2,5,6 GB RAM,128 GB ROM,64MP Rear Camera,NaN,4500 mAh,1 Year Warranty,"₹27,899",6/17/2023


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1828 entries, 0 to 1827
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Phone Name         1828 non-null   object 
 1   Rating ?/5         1828 non-null   float64
 2   Number of Ratings  1828 non-null   object 
 3   RAM                1828 non-null   object 
 4   ROM/Storage        1661 non-null   object 
 5   Back/Rare Camera   1820 non-null   object 
 6   Front Camera       1435 non-null   object 
 7   Battery            1815 non-null   object 
 8   Processor          1780 non-null   object 
 9   Price in INR       1828 non-null   object 
 10  Date of Scraping   1828 non-null   object 
dtypes: float64(1), object(10)
memory usage: 157.2+ KB


Majority of the columns are object, have to convert in int


In [6]:
display(df.isnull().sum())


,0
Phone Name,0
Rating ?/5,0
Number of Ratings,0
RAM,0
ROM/Storage,167
Back/Rare Camera,8
Front Camera,393
Battery,13
Processor,48
Price in INR,0


In [7]:
df.shape

(1828, 11)

# Observation Yet
1. Date of scrapping could be drop.
2. columns could be rename to short
3. units could be removed from Storage, Camera, Price and battery.
4. about 45% data loss in front camera.
5. about 10% data is missing in ROM/Storage, about 3% in processor.
6. rows with missing battery could be deleted in needed.
7. rare camera could be split into three cols.

In [39]:
demo=df.copy()

In [40]:
demo.rename(columns={'Phone Name': 'Phone','Rating ?/5': 'Rating','ROM/Storage':'ROM(GB)','RAM':'RAM(GB)','Front Camera':'FrontCam(MP)','Back/Rare Camera':'RareCam(MP)','Battery':'Battery(in mah)','Price':'Price in INR'}, inplace=True)

In [10]:
display(demo.columns)

Index(['Phone', 'Rating', 'Number of Ratings', 'RAM(GB)', 'ROM(GB)',
       'RareCam(MP)', 'FrontCam(MP)', 'Battery(in mah)', 'Processor',
       'Price in INR', 'Date of Scraping'],
      dtype='object')

In [41]:
demo.drop(columns=['Date of Scraping'],inplace=True)

In [12]:
demo.sample(3)

,Phone,Rating,Number of Ratings,RAM(GB),ROM(GB),RareCam(MP),FrontCam(MP),Battery(in mah),Processor,Price in INR
566,"SAMSUNG Galaxy A34 5G (Awesome Silver, 128 GB)",4.2,715,8 GB RAM,128 GB ROM,48MP + 8MP + 5MP,13MP Front Camera,5000 mAh,"Dimensity 1080, Octa Core Processor","₹30,999"
389,"APPLE iPhone 14 Pro Max (Gold, 128 GB)",4.6,"1,466",128 GB ROM,NaN,48MP + 12MP + 12MP,12MP Front Camera,A16 mAh,1 Year Warranty for Phone and 6 Months Warrant...,"₹1,27,999"
57,"POCO C31 (Royal Blue, 32 GB)",4.3,"1,45,794",3 GB RAM,32 GB ROM,13MP + 2MP + 2MP,5MP Front Camera,5000 mAh,MediaTek Helio G35 Processor,"₹8,499"


# Removing Units and converting Columns into integers

In [42]:
demo['Price in INR'] = demo['Price in INR'].str.replace('₹', '').str.replace(',', '').astype(int)

In [43]:
demo['Number of Ratings']=demo['Number of Ratings'].str.replace(',','').astype(int)

In [44]:
demo['FrontCam(MP)'] = demo['FrontCam(MP)'].astype(str).str.replace('MP Front Camera','', regex=False).str.replace('MP + 2MP Dual Front Camera','', regex=False).str.replace('nan','0', regex=False).str.replace('NaN','0', regex=False).str.strip()
demo['FrontCam(MP)'] = pd.to_numeric(demo['FrontCam(MP)'], errors='coerce').fillna(0).astype(int)

In [45]:
demo['ROM(GB)']=demo['ROM(GB)'].str.replace('GB ROM','',regex=False).str.strip()
demo['ROM(GB)']=pd.to_numeric(demo['ROM(GB)'],errors='coerce')
demo.dropna(subset=['ROM(GB)'],inplace=True)
demo['ROM(GB)']=demo['ROM(GB)'].astype(int)

In [48]:
demo['ROM(GB)']=demo['ROM(GB)'].str.replace('GB ROM','',regex=False).str.strip()
demo['ROM(GB)']=pd.to_numeric(demo['ROM(GB)'],errors='coerce')
demo.dropna(subset=['ROM(GB)'],inplace=True)
demo['ROM(GB)']=demo['ROM(GB)'].astype(int)

AttributeError: Can only use .str accessor with string values!

In [49]:
demo['RAM(GB)']=demo['RAM(GB)'].str.replace('GB RAM','',regex=False).str.strip()
demo['RAM(GB)']=pd.to_numeric(demo['RAM(GB)'],errors='coerce')
demo.dropna(subset=['RAM(GB)'],inplace=True)
demo['RAM(GB)']=demo['RAM(GB)'].astype(int)

AttributeError: Can only use .str accessor with string values!

In [50]:
demo['Battery(in mah)']=demo['Battery(in mah)'].str.replace('mAh','',regex=False).str.strip()
demo['Battery(in mah)']=pd.to_numeric(demo['Battery(in mah)'],errors='coerce')
demo.dropna(subset=['Battery(in mah)'],inplace=True)
demo['Battery(in mah)']=demo['Battery(in mah)'].astype(int)

In [51]:
demo.sample(3)

,Phone,Rating,Number of Ratings,RAM(GB),ROM(GB),RareCam(MP),FrontCam(MP),Battery(in mah),Processor,Price in INR
1134,"Redmi 10 Prime (Phantom Black, 64 GB)",4.1,134,4,64,8MP Rear Camera,0,6000,1 Year Manufacturer Warranty for Handset and 6...,12999
65,"POCO C3 (Matte Black, 32 GB)",4.3,400433,3,32,13MP + 2MP + 2MP,5,5000,Mediatek Helio G35 Processor,9999
228,"realme Narzo 30 (Racing Blue, 64 GB)",4.3,77321,6,64,48MP + 2MP + 2MP,16,5000,MediaTek Helio G95 Processor,11499


In [24]:
display(demo.isnull().sum())

,0
Phone,0
Rating,0
Number of Ratings,0
RAM(GB),0
ROM(GB),0
RareCam(MP),0
FrontCam(MP),0
Battery(in mah),0
Processor,3
Price in INR,0


In [25]:
demo.shape

(1550, 10)

# Splitting RareCam(MP) into three Columns

In [52]:
def split_rare_camera(camera_str):
    if pd.isna(camera_str):
        return [0, 0, 0]
    # Remove non-numeric characters except '+' and split by '+'
    camera_str = camera_str.replace('MP', '').replace('Rear Camera', '').replace('Dual Camera', '').replace('AI Lens Camera', '').strip()
    cameras = camera_str.split('+')

    result = [0, 0, 0]
    for i, cam in enumerate(cameras[:3]):
        try:
            result[i] = int(cam.strip())
        except ValueError:
            result[i] = 0
    return result

demo[['RCam1(MP)', 'RCam2(MP)', 'RCam3(MP)']] = demo['RareCam(MP)'].apply(lambda x: pd.Series(split_rare_camera(x)))

display(demo[['RareCam(MP)', 'RCam1(MP)', 'RCam2(MP)', 'RCam3(MP)']].head())

,RareCam(MP),RCam1(MP),RCam2(MP),RCam3(MP)
0,8MP Dual Camera,8,0,0
1,50MP + 2MP,50,2,0
2,8MP Dual Rear Camera,0,0,0
3,50MP Dual Rear Camera,0,0,0
4,8MP Dual Rear Camera,0,0,0


In [53]:
demo.shape

(1550, 13)

In [54]:
demo.sample(4)

,Phone,Rating,Number of Ratings,RAM(GB),ROM(GB),RareCam(MP),FrontCam(MP),Battery(in mah),Processor,Price in INR,RCam1(MP),RCam2(MP),RCam3(MP)
1580,"OnePlus Nord CE 5G (Charcoal Ink, 128 GB)",4.2,5,6,128,64MP Rear Camera,0,4500,1 Year Warranty,22990,64,0,0
129,"realme C33 2023 (Aqua Blue, 128 GB)",4.4,13111,4,128,50MP + 0.3MP,5,5000,Unisoc T612 Processor,10499,50,0,0
767,"OPPO K10 (Black Carbon, 128 GB)",4.4,110994,6,128,50MP + 2MP + 2MP,16,5000,Qualcomm Snapdragon 680 Processor,14990,50,2,2
207,"realme Narzo 50A (Oxygen Blue, 64 GB)",4.4,342411,4,64,50MP + 2MP + 2MP,8,6000,MediaTek Helio G85 Processor,11499,50,2,2


In [55]:
demo.dropna(subset=['Processor'],inplace=True)

In [56]:
demo.drop(columns=['RareCam(MP)'],inplace=True)

In [57]:
demo.sample(4)

,Phone,Rating,Number of Ratings,RAM(GB),ROM(GB),FrontCam(MP),Battery(in mah),Processor,Price in INR,RCam1(MP),RCam2(MP),RCam3(MP)
508,"SAMSUNG Galaxy F13 (Waterfall Blue, 64 GB)",4.3,151074,4,64,8,6000,Exynos 850 Processor,10999,50,5,2
1003,"vivo Y02 (Orchid Blue, 32 GB)",4.3,542,3,32,5,5000,MediaTek P22 Processor,12999,8,0,0
1513,"MOTOROLA E7 Power (Tahiti Blue, 32 GB)",4.1,10429,2,32,5,5000,MediaTek Helio G25 Processor,6999,13,2,0
961,"vivo V27 Pro 5G (Magic Blue, 128 GB)",4.4,5070,8,128,50,4600,Mediatek Dimensity 8200 Processor,37999,0,8,2


In [58]:
demo.isnull().sum()

,0
Phone,0
Rating,0
Number of Ratings,0
RAM(GB),0
ROM(GB),0
FrontCam(MP),0
Battery(in mah),0
Processor,0
Price in INR,0
RCam1(MP),0


In [59]:
demo.shape

(1547, 12)

In [62]:
demo.to_csv('processed_mobile_prices.csv', index=False)

In [63]:
from google.colab import files
files.download('processed_mobile_prices.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# NEW Data set is Created to apply ML Algorithms